In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
from sklearn.preprocessing import MinMaxScaler

In [ ]:

data_path = r"D:\displaced prediction\data\###.xlsx"
df = pd.read_excel(data_path, index_col=0)

df.fillna(0, inplace=True)

target_column = df.iloc[:, 0].values.reshape(-1, 1)
features = df.iloc[:, 0:].values

scaler_target = MinMaxScaler()
scaler_features = MinMaxScaler()

target_scaled = scaler_target.fit_transform(target_column)
features_scaled = scaler_features.fit_transform(features)

window = 3 
train_ratio = 0.8 

def create_sequences(features, target, window):
    X, y = [], []
    for i in range(len(features) - window):
        X.append(features[i:i + window])
        y.append(target[i + window])
    return np.array(X), np.array(y)

X, y = create_sequences(features_scaled, target_scaled, window)

train_size = int(len(X) * train_ratio)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

svr = SVR(kernel='rbf', C=60, gamma=0.05, epsilon=0.05)
svr.fit(X_train, y_train.ravel())

y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)

y_train_actual = scaler_target.inverse_transform(y_train)
y_test_actual = scaler_target.inverse_transform(y_test)
y_train_pred_actual = scaler_target.inverse_transform(y_train_pred.reshape(-1, 1))
y_test_pred_actual = scaler_target.inverse_transform(y_test_pred.reshape(-1, 1))

true = y_test_actual
pre = y_test_pred_actual
mse = np.sum((true - pre) ** 2) / len(pre)
rmse = np.sqrt(mse)
mae = np.sum(np.absolute(true - pre)) / len(pre)
r2 = 1 - mse / np.var(pre)
mape = np.mean(np.abs((pre - true) / true))

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)
print("MAPE:", mape)

plt.figure(figsize=(10, 4), dpi=150)
plt.plot(y_test_actual, label="Actual", color='red', linewidth=4)
plt.plot(y_test_pred_actual, color='blue', label='Prediction', linewidth=2.5, linestyle="--")
plt.title('SVR Prediction', size=15)
plt.ylabel('AQI', size=15)
plt.xlabel('Time', size=15)
plt.legend()
plt.show()


In [ ]:

results = pd.DataFrame({
    'Actual': y_test_actual.flatten(),
    'Prediction': y_test_pred_actual.flatten()
})
results.to_excel(r"C:\Users\10512\Downloads\1.xlsx", index=False)

print("预测结果已导出到文件中")